In [11]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

import datetime
import io

from PIL import Image
from IPython.display import Image as IPyImage
import imageio

print("Version: ", tf.__version__)
tf.get_logger().setLevel('INFO')

Version:  2.15.0


In [12]:
def get_model():
  model = tf.keras.Sequential()
  model.add(tf.keras.layers.Dense(1, activation='linear', input_dim=784))
  model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.1), loss='mean_squared_error', metrics=['mae'])
  return model

In [15]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000, 784).astype('float32') / 255
x_test = x_test.reshape(10000, 784).astype('float32') / 255

In [16]:
class MyCustomCallback(tf.keras.callbacks.Callback):
  def on_train_batch_begin(self, batch, logs=None):
    print('Training: batch {} begins at {}'.format(batch, datetime.datetime.now().time()))

  def on_train_batch_end(self, batch, logs=None):
    print('Training: batch {} begins at {}'.format(batch, datetime.datetime.now().time()))

In [18]:
model = get_model()
_ = model.fit(x_train, y_train,
          batch_size=64,
          epochs=1,
          steps_per_epoch=5,
          verbose=0,
          callbacks=[MyCustomCallback()])

Training: batch 0 begins at 08:02:45.193293
Training: batch 0 begins at 08:02:45.608685
Training: batch 1 begins at 08:02:45.610928
Training: batch 1 begins at 08:02:45.613451
Training: batch 2 begins at 08:02:45.614489
Training: batch 2 begins at 08:02:45.617722
Training: batch 3 begins at 08:02:45.619022
Training: batch 3 begins at 08:02:45.622048
Training: batch 4 begins at 08:02:45.623560
Training: batch 4 begins at 08:02:45.626734


### An overview of callback methods